# Acinetobacter baumannii mass spectrometry data mapping

Using multiple metabolite databases to elucidate structures obtained with mass spectrometry. Data was obtained from " Untargeted metabolomics analysis reveals key pathways responsible for the synergistic killing of colistin and doripenem combination against Acinetobacter baumannii".

In [ ]:
# Importing packages
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import string

# COBRA toolbox specific packages
from cobra import Model, Reaction, Metabolite
import cobra
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

In [ ]:
# Loading mass spec. data into dataframe
df_AB_mass_spec_data = pd.read_csv("Acinetobacter_baumannii_mass_spec.csv")
df_AB_mass_spec_data_important_features = df_AB_mass_spec_data.loc[:, 'Mass':'Max intensity'] # extracting important features
# df_AB_mass_spec_data_important_features.head()

df_AB_mass_spec_minimal = df_AB_mass_spec_data_important_features[['Mass','Formula','Putative metabolite']] # Mass, formula, name
df_AB_mass_spec_minimal.head()

In [ ]:
df_AB_mass_spec_data

# A. baumannii network reconstruction
From "iCN718, an Updated and Improved Genome-Scale Metabolic Network Reconstruction of Acinetobacter baumannii AYE"

In [ ]:
# Loading Acinetobacter baumannii network reconstruction 'iCN718'
AB_model=cobra.io.load_json_model('iCN718.json')
AB_model.metabolites.atp_c.formula

# Mass calculator 

In [ ]:
# Writing function that calculates molar mass from chemical formula
def calc_mass_from_formula( formula ):
    
    # Typical mass of compounds --- assumes non-monoisotopic
    C = 12.0107 # +/- 0.0008
    H = 1.00794 # +/- 0.00001
    N = 14.0067 # +/- 0.0001
    O = 15.9994 # +/-
    P = 30.973762 # +/- 0.000002
    S =  32.065 # +/- 0.005

    # Initializing new string for first round of alteration (i.e., adding the '+')
    new_str = ''

    # Adding a '+' before every element
    for index in range (0, len(formula)):
        temp_str = formula[index]
        if temp_str.isalpha():
            new_str = new_str + '+' + temp_str 
        elif temp_str.isnumeric():
            new_str = new_str + temp_str
            
    # Removing the '+' at teh beginning of each string
    if new_str[0] == '+':
        str_formula_add = new_str[1:]

    # Initializing new string for second round of alteration (i.e., adding the '*')
    final_str_formula = ''

    # Adding a '*' after each element that is followed by a number
    for index in range(0,len(str_formula_add)):
        temp_str_1 = str_formula_add[index]
        if index != len(str_formula_add)-1:
            temp_str_2 = str_formula_add[index + 1]
        if temp_str_1.isalpha() and temp_str_2.isnumeric():
            final_str_formula = final_str_formula + temp_str_1 + '*'
        else: 
            final_str_formula = final_str_formula + temp_str_1

    # Evaludating string expression and returning molecular mass to user
    molecular_mass = eval(final_str_formula)    
    return molecular_mass



In [ ]:
# Test case of function (output should be 189.1659 )
mass = calc_mass_from_formula('C7H11NO5')
print(mass)

# Parsing A. baumannii - specific database (from biocyc)


In [ ]:
# Parsing data A_Baumannii file containing compounds

with open('A_Baumannii_compounds.txt') as f:
    # counter = 0
    # common_name = []
    # inchi = []
    # mono_mw = []
    start = False
    compounds = []
    for line in f:
        if line.strip().split()[0] =='UNIQUE-ID':
            start = True
            compound = {'COMMON-NAME':'','INCHI':'', 'MONOISOTOPIC-MW':0}
        if line.strip() == '//':
            start = False
            compounds.append(list(compound.values()))
        if start:
            line = line.strip().split()
            if line[0] == 'COMMON-NAME':
                compound['COMMON-NAME'] = ''.join(line[2:])
            if line[0] == 'INCHI':
                compound['INCHI'] = line[2].split('/')[1]
            if line[0] == 'MONOISOTOPIC-MW':
                compound['MONOISOTOPIC-MW'] = line[2]
    print(len(compounds))                    
    print(compounds[0])

In [ ]:
len(compounds)


In [ ]:
# Create new dataframe for comparison
df_AB_mass_spec_mapping = df_AB_mass_spec_minimal.copy()
df_AB_mass_spec_mapping['mapped_MW'] = ''
df_AB_mass_spec_mapping['mapped_formula'] = ''
df_AB_mass_spec_mapping['mapped_metabs'] = ''

df_AB_mass_spec_mapping

# df_AB_mass_spec_mapping.loc[1,'mapped_MW'] = 'test'
# df_AB_mass_spec_mapping

In [ ]:
# List of candidates 
# ppm = mass error / exact mass * 10^6

counter = 0

for mass_measured in df_AB_mass_spec_mapping['Mass']:
    candidate_metabs_mass = []
    candidate_metabs_formula = []
    candidate_metabs_name = []
    for compound in compounds:
        
        candidate_mass = compound[0]
        candidate_formula = compound[1]
        candidate_name = compound[2]
        
        if float(candidate_mass) > 0.1:
            error_ppm =  (abs((float(candidate_mass) - mass_measured)) / float(candidate_mass)) * (10**6)
        else:
            error_ppm = 1000000
        
        # Applying 5ppm error threshold 
        if error_ppm < 25:
            candidate_metabs_mass.append(candidate_mass)
            candidate_metabs_formula.append(candidate_formula)
            candidate_metabs_name.append(candidate_name)
            # print('got one') - verification that threshold is working to work
            
    candidate_metabs_mass_comb = ', '.join(map(str,candidate_metabs_mass))
    candidate_metabs_formula_comb = ', '.join(map(str,candidate_metabs_formula))
    candidate_metabs_name_comb = ', '.join(map(str,candidate_metabs_name))
    
#     if len(candidate_metabs_mass_comb) > 2:
#         test =candidate_metabs_mass
    
    df_AB_mass_spec_mapping.loc[counter,'mapped_MW'] = candidate_metabs_mass_comb
    df_AB_mass_spec_mapping.loc[counter,'mapped_formula'] = candidate_metabs_formula_comb
    df_AB_mass_spec_mapping.loc[counter,'mapped_metabs'] = candidate_metabs_name_comb
    
    counter += 1


# Displaying mapped dataframe
df_AB_mass_spec_mapping   

In [ ]:
# Creating list with number of mapped matches

metab_matches = []

for mapped_metab in df_AB_mass_spec_mapping['mapped_metabs']:
    # print(mapped_metab)
    if len(mapped_metab) == 0:
        metab_matches.append(len(mapped_metab))
    else: 
        temp_metabs = mapped_metab.split(', ')
        temp_matches = len(temp_metabs)
        metab_matches.append(temp_matches)

In [ ]:
metab_matches.count(0)

In [ ]:


fig = plt.figure(figsize=(12, 9))  

ax = plt.subplot(111)  
ax.spines["top"].set_visible(False)  
ax.spines["right"].set_visible(False) 

ax.get_xaxis().tick_bottom()  
ax.get_yaxis().tick_left() 

plt.xticks(fontsize=14)  
plt.yticks(range(0, 1347, 200), fontsize=14) 

plt.xlabel("Number of metabolite matches", fontsize=20)  
plt.ylabel("Occurences", fontsize=20)

plt.title('Matches in metabolite mapping', fontsize=26)

plt.hist(metab_matches,  
         color="#3F5D7D", bins=11) 
fig.savefig('test.png')
plt.show()

fig.savefig('test.png')

In [ ]:
# Bring 0-matches value down --- look for databases on KEGG and HMDB

LCMS_meausred_masses = df_AB_mass_spec_mapping['Mass']
type(LCMS_meausred_masses)
LCMS_meausred_masses.to_csv('LCMS_meausred_masses.csv')

# Using MetCyc database

In [ ]:
# Parsing data A_Baumannii file containing compounds

with open('compounds.dat') as f:
    # counter = 0
    # common_name = []
    # inchi = []
    # mono_mw = []
    start = False
    compounds_MC = []
    for line in f:
        if line.strip().split()[0] =='UNIQUE-ID':
            start = True
            compound_MC = {'COMMON-NAME':'','INCHI':'', 'MONOISOTOPIC-MW':0}
        if line.strip() == '//':
            start = False
            compounds_MC.append(list(compound_MC.values()))
        if start:
            line = line.strip().split()
            if line[0] == 'COMMON-NAME':
                compound_MC['COMMON-NAME'] = ''.join(line[2:])
            if line[0] == 'INCHI':
                compound_MC['INCHI'] = line[2].split('/')[1]
            if line[0] == 'MONOISOTOPIC-MW':
                compound_MC['MONOISOTOPIC-MW'] = line[2]
    # print(len(compounds))                    
    # print(compounds[0])

In [ ]:
len(compounds_MC)

In [ ]:
# Create new dataframe for comparison
df_AB_mass_spec_mapping_MC = df_AB_mass_spec_minimal.copy()
df_AB_mass_spec_mapping_MC['mapped_MW'] = ''
df_AB_mass_spec_mapping_MC['mapped_formula'] = ''
df_AB_mass_spec_mapping_MC['mapped_metabs'] = ''

df_AB_mass_spec_mapping_MC

# df_AB_mass_spec_mapping.loc[1,'mapped_MW'] = 'test'
# df_AB_mass_spec_mapping

In [ ]:
# List of candidates 
# ppm = mass error / exact mass * 10^6

counter = 0

for mass_measured_MC in df_AB_mass_spec_mapping_MC['Mass']:
    candidate_metabs_mass_MC = []
    candidate_metabs_formula_MC = []
    candidate_metabs_name_MC = []
    for compound_MC in compounds_MC:
        
        candidate_mass_MC = compound_MC[0]
        candidate_formula_MC = compound_MC[1]
        candidate_name_MC = compound_MC[2]
        
        if float(candidate_mass_MC) > 0.1:
            error_ppm =  (abs((float(candidate_mass_MC) - mass_measured_MC)) / float(candidate_mass_MC)) * (10**6)
        else:
            error_ppm = 1000000
        
        # Applying 5ppm error threshold 
        if error_ppm < 25:
            candidate_metabs_mass_MC.append(candidate_mass_MC)
            candidate_metabs_formula_MC.append(candidate_formula_MC)
            candidate_metabs_name_MC.append(candidate_name_MC)
            # print('got one') - verification that threshold is working to work
            
    candidate_metabs_mass_comb_MC = ', '.join(map(str,candidate_metabs_mass_MC))
    candidate_metabs_formula_comb_MC = ', '.join(map(str,candidate_metabs_formula_MC))
    candidate_metabs_name_comb_MC = ', '.join(map(str,candidate_metabs_name_MC))
    
#     if len(candidate_metabs_mass_comb) > 2:
#         test =candidate_metabs_mass
    
    df_AB_mass_spec_mapping_MC.loc[counter,'mapped_MW'] = candidate_metabs_mass_comb_MC
    df_AB_mass_spec_mapping_MC.loc[counter,'mapped_formula'] = candidate_metabs_formula_comb_MC
    df_AB_mass_spec_mapping_MC.loc[counter,'mapped_metabs'] = candidate_metabs_name_comb_MC
    
    counter += 1


# Displaying mapped dataframe
df_AB_mass_spec_mapping_MC

In [ ]:
# Creating list with number of mapped matches

metab_matches_MC = []

for mapped_metab_MC in df_AB_mass_spec_mapping_MC['mapped_metabs']:
    # print(mapped_metab)
    if len(mapped_metab_MC) == 0:
        metab_matches_MC.append(len(mapped_metab_MC))
    else: 
        temp_metabs_MC = mapped_metab_MC.split(', ')
        temp_matches_MC = len(temp_metabs_MC)
        metab_matches_MC.append(temp_matches_MC)

In [ ]:
metab_matches_MC.index(74)
df_AB_mass_spec_mapping_MC.loc[118, "mapped_metabs"]

In [ ]:
fig = plt.figure(figsize=(12, 9))  

ax = plt.subplot(111)  
ax.spines["top"].set_visible(False)  
ax.spines["right"].set_visible(False) 

ax.get_xaxis().tick_bottom()  
ax.get_yaxis().tick_left() 

plt.xticks(fontsize=14)  
plt.yticks(range(0, 650, 200), fontsize=14) 

plt.xlabel("Number of metabolite matches", fontsize=20)  
plt.ylabel("Occurences", fontsize=20)

plt.title('Matches in metabolite mapping', fontsize=26)

plt.hist(metab_matches_MC,  
         color="#3F5D7D",bins = 75) 
fig.savefig('test2.png')
plt.show()
fig.savefig('test2.png')

# BioCyc + MetCyc

In [ ]:
compounds_mixed = compounds + compounds_MC

compounds_mixed

In [ ]:
compounds == compounds_MC

In [ ]:
# Create new dataframe for comparison
df_AB_mass_spec_mapping_mixed = df_AB_mass_spec_minimal.copy()
df_AB_mass_spec_mapping_mixed['mapped_MW'] = ''
df_AB_mass_spec_mapping_mixed['mapped_formula'] = ''
df_AB_mass_spec_mapping_mixed['mapped_metabs'] = ''

df_AB_mass_spec_mapping_mixed

# df_AB_mass_spec_mapping.loc[1,'mapped_MW'] = 'test'
# df_AB_mass_spec_mapping

In [ ]:
# List of candidates 
# ppm = mass error / exact mass * 10^6

counter = 0

for mass_measured_mixed in df_AB_mass_spec_mapping_mixed['Mass']:
    candidate_metabs_mass_mixed = []
    candidate_metabs_formula_mixed = []
    candidate_metabs_name_mixed = []
    for compound_mixed in compounds_mixed:
        
        candidate_mass_mixed = compound_mixed[0]
        candidate_formula_mixed = compound_mixed[1]
        candidate_name_mixed = compound_mixed[2]
        
        if float(candidate_mass_mixed) > 0.1:
            error_ppm =  (abs((float(candidate_mass_mixed) - mass_measured_mixed)) / float(candidate_mass_mixed)) * (10**6)
        else:
            error_ppm = 1000000
        
        # Applying 5ppm error threshold 
        if error_ppm < 25:
            candidate_metabs_mass_mixed.append(candidate_mass_mixed)
            candidate_metabs_formula_mixed.append(candidate_formula_mixed)
            candidate_metabs_name_mixed.append(candidate_name_mixed)
            # print('got one') - verification that threshold is working to work
            
    candidate_metabs_mass_comb_mixed = ', '.join(map(str,candidate_metabs_mass_mixed))
    candidate_metabs_formula_comb_mixed = ', '.join(map(str,candidate_metabs_formula_mixed))
    candidate_metabs_name_comb_mixed = ', '.join(map(str,candidate_metabs_name_mixed))
    
#     if len(candidate_metabs_mass_comb) > 2:
#         test =candidate_metabs_mass
    
    df_AB_mass_spec_mapping_mixed.loc[counter,'mapped_MW'] = candidate_metabs_mass_comb_mixed
    df_AB_mass_spec_mapping_mixed.loc[counter,'mapped_formula'] = candidate_metabs_formula_comb_mixed
    df_AB_mass_spec_mapping_mixed.loc[counter,'mapped_metabs'] = candidate_metabs_name_comb_mixed
    
    counter += 1


# Displaying mapped dataframe
df_AB_mass_spec_mapping_mixed

In [ ]:
# Creating list with number of mapped matches

metab_matches_mixed = []

for mapped_metab_mixed in df_AB_mass_spec_mapping_mixed['mapped_metabs']:
    # print(mapped_metab)
    if len(mapped_metab_mixed) == 0:
        metab_matches_mixed.append(len(mapped_metab_mixed))
    else: 
        temp_metabs_mixed = mapped_metab_mixed.split(', ')
        temp_matches_mixed = len(temp_metabs_mixed)
        metab_matches_mixed.append(temp_matches_mixed)

In [ ]:
max(metab_matches_mixed)

In [ ]:
plt.figure(figsize=(12, 9))  

ax = plt.subplot(111)  
ax.spines["top"].set_visible(False)  
ax.spines["right"].set_visible(False) 

ax.get_xaxis().tick_bottom()  
ax.get_yaxis().tick_left() 

plt.xticks(fontsize=14)  
plt.yticks(range(0, 650, 200), fontsize=14) 

plt.xlabel("Number of metabolite matches", fontsize=16)  
plt.ylabel("Occurences", fontsize=16)

plt.title('Matches in metabolite mapping', fontsize=24)

plt.hist(metab_matches_mixed,  
         color="#3F5D7D",bins = 81) 

plt.show()

# Less 0 matches, but also less singular matches 

# Pathway determination 

In [ ]:
# Creating new dataframe with pathways
df_AB_mass_spec_mapping_pathways = df_AB_mass_spec_mapping.copy()
df_AB_mass_spec_mapping_pathways['pathway'] = ''

df_AB_mass_spec_mapping_pathways['pathway'] = df_AB_mass_spec_data.loc[:, 'Pathway']
df_AB_mass_spec_mapping_pathways['mapped_metabs'].loc[1693]



In [ ]:
df_AB_mass_spec_mapping_pathways

In [ ]:
# Creating set of pathways

pathway_list = []
for pathway in df_AB_mass_spec_mapping_pathways['pathway']:
    temp_pathways = str(pathway)
    if '__' in temp_pathways: # '__' was used to split pathways in Excel spreadsheet
        pathway_split = temp_pathways.split('__')
        for index in range(len(pathway_split)):
            pathway_list.append(pathway_split[index])
    else:
        pathway_list.append(temp_pathways)
pathway_set = set(pathway_list)
        
# Cleaning-up set (removing missed case with multiple pathway and removing empty space if present)
pathway_list_final = []
for pathway in pathway_set:
    if pathway == 'Glutathione metabolism_Butanoate metabolism_C5-Branched dibasic acid metabolism_Porphyrin and chlorophyll metabolism_Nitrogen metabolism':
        print('removed') # tell me when missed case is removed
    elif pathway[0] == ' ':
        if pathway[-1] == ' ':
            pathway_list_final.append(pathway[1:-1])
        else: 
            pathway_list_final.append(pathway[1:])
    elif pathway[-1] == ' ':
        if pathway[0] == ' ':
            pathway_list_final.append(pathway[1:-1])
        else: 
            pathway_list_final.append(pathway[:-1])        
    else:
        pathway_list_final.append(pathway)
        
pathway_set_final = sorted(set(pathway_list_final))


# Creating a set of pathway stemming from the original data set
# pathway_list = []
# for pathway in df_AB_mass_spec_mapping_pathways['pathway']:
#     temp_pathways = str(pathway)
#     if '__' in temp_pathways: # '__' was used to split pathways in Excel spreadsheet
#         pathway_split = temp_pathways.split('__')
#         for index in range(len(pathway_split)):
#             pathway_list.append(pathway_split[index])
#     else:
#         pathway_list.append(temp_pathways)
# pathway_set = set(pathway_list)
        
# # Cleaning-up set (removing missed case with multiple pathway and removing empty space if present)
# pathway_list_final = []
# for pathway in pathway_set:
#     if pathway == 'Glutathione metabolism_Butanoate metabolism_C5-Branched dibasic acid metabolism_Porphyrin and chlorophyll metabolism_Nitrogen metabolism':
#         print('removed') # tell me when missed case is removed
#     elif pathway[0] == ' ':
#         pathway_list_final.append(pathway[1:])
#     else:
#         pathway_list_final.append(pathway)
        
# pathway_set_final = sorted(set(pathway_list_final))

    

In [ ]:
pathway_set_final

In [ ]:
# Counting pathways and how many were missed 

pathway_analysis_list = []

for pathway in pathway_set_final:
    Actual_instances = 0
    Measured_instances = 0
    counter = 0
    
    pathway_dict = {'pathway':pathway,'Actual instances':0, 'Meausred instances':0, 'Percentage_captured':0}
    for pathway_options in df_AB_mass_spec_mapping_pathways['pathway']:
        if pathway in pathway_options:
            Actual_instances += 1
            if df_AB_mass_spec_mapping_pathways.loc[counter,'mapped_formula'] == '':
                pass
            else:
                Measured_instances += 1
            counter += 1
    
    pathway_dict['Actual instances'] = int(Actual_instances)
    pathway_dict['Meausred instances'] = int(Measured_instances)
    pathway_dict['Percentage_captured'] = float(Measured_instances)/float(Actual_instances) * 100 
    
    pathway_analysis_list.append(pathway_dict)


In [ ]:
# Prepping pathway analysis to csv export

pathway_analysis_df = pd.DataFrame()
for index in range(len(pathway_analysis_list)):
    pathway_analysis_df = pathway_analysis_df.append(pathway_analysis_list[index], ignore_index = True)
# pathway_analysis_df

cols = pathway_analysis_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
pathway_analysis_df = pathway_analysis_df[cols]
pathway_analysis_df

In [ ]:
# pathway_analysis_df.to_csv('pathway_analysis_final.csv') 
# Most missing metabolites in unknown pathways


In [ ]:
# Looking at metabolites associated with underrepresented pathways 
counter = 0
missing_metabs = []
for pathway in df_AB_mass_spec_mapping_pathways['pathway']:
    if pathway == '0':
        missing_metabs.append(df_AB_mass_spec_mapping_pathways.loc[counter,'Putative metabolite'])
    counter += 1
        
missing_metabs        
        
        
        

# Populating A. Baumannii reconstruction with metabolite formulas


In [ ]:
# Loading Acinetobacter baumannii network reconstruction 'iCN718' and E.coli model iJO1366
AB_model = cobra.io.load_json_model('iCN718.json')
Ecoli_model = cobra.io.load_json_model('iJO1366.json')

# Ecoli_model.metabolites.get_by_id('atp_c')


In [ ]:
missing = []
for metabolite_AB in AB_model.metabolites:
    temp = metabolite_AB.id
    a = False
    for metabolite_EC in Ecoli_model.metabolites:
        temp2 = metabolite_EC.id
        if AB_model.metabolites.get_by_id(temp).id == Ecoli_model.metabolites.get_by_id(temp2).id:
            a = True
            AB_model.metabolites.get_by_id(temp).formula =  Ecoli_model.metabolites.get_by_id(temp2).formula
    if a == False:
        print(temp)
        missing.append(temp)
                
                

In [ ]:
AB_S_matrix = cobra.util.array.create_stoichiometric_matrix(AB_model)
AB_S_matrix

In [ ]:
import networkx as nx

AB_adjacency = (np.dot(AB_S_matrix, AB_S_matrix.T) > 0).astype(int)
AB_adjacency

# graph = nx.from_numpy_matrix(AB_adjacency)
# graph

# nx.draw(G)
fig = plt.figure(figsize=(12, 9)) 
plt.figure(figsize=(100,100))

G = nx.from_numpy_matrix(np.array(AB_adjacency), create_using=nx.MultiDiGraph())
# pos = nx.circular_layout(G)
pos = nx.random_layout(G)

# nx.draw_circular(G)
nx.draw_random(G)
labels = {i : i + 1 for i in G.nodes()}
nx.draw_networkx_labels(G, pos, labels, font_size=15)

plt.figure

plt.figure(figsize=(300,400))
fig.savefig('test3.png')
plt.show()

fig.savefig('test3.png')

In [ ]:
for rxn in AB_model.reactions:
    
    if rxn.subsystem == 'Glycolysis/ Gluconeogenesis':
        print(rxn)